In [1]:
import trainer
from utils import batchify
import json
import tensorflow as tf
import numpy as np

In [2]:
VERSION = '107_intent'
with open(VERSION + '/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open(VERSION + '/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)

In [3]:
with open(VERSION + '/trainer_params.json', 'r') as inp:
    params = json.load(inp)
my_trainer = trainer.Trainer(**params, fine_tune=True)
my_trainer.logger.info('Trainer params {}'.format(params))
tf.reset_default_graph()
my_trainer.build(save_optimizer_var=False)

Trainer params {'model_configs': {'drop_e': 0.1, 'skip_connection': True, 'projection_dims': 512, 'char_cnn_options': {'n_highways': 2, 'layers': [[1, 16], [2, 16], [3, 32], [4, 64], [5, 128], [6, 256], [7, 512]]}, 'char_vocab_size': 1088, 'vocab_size': 125409, 'rnn_layers': [{'drop_o': 0.2, 'drop_i': 0.2, 'units': 1024, 'wdrop': 0.4}, {'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4}, {'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4}], 'char_vec_size': 16}, 'bptt': 100, 'optimizer': {'params': {'weight_decay': 1.2e-06, 'beta2': 0.99, 'beta1': 0.8}, 'name': 'adamw'}, 'train_summary_dir': '107_intent/train_summary/', 'negative_samples': 10000, 'wdecay': 0.0, 'checkpoint_dir': '107_intent/checkpoints/', 'log_path': '107_intent/logs', 'beta': 1e-06, 'use_ema': True, 'test_summary_dir': '107_intent/test_summary/', 'clip_norm': 0.3, 'save_freq': 1000, 'alpha': 1e-06}


INFO:tensorflow:Restoring parameters from 107_intent/checkpoints/train/model.cpkt-73000


Restoring parameters from 107_intent/checkpoints/train/model.cpkt-73000


In [4]:
with open('IntentDetection/full/lm_train_word.npy','rb') as inp:
    train_word = np.load(inp)
with open('IntentDetection/full/lm_train_char.npy','rb') as inp:
    train_char = np.load(inp)
with open('IntentDetection/full/lm_valid_word.npy','rb') as inp:
    valid_word = np.load(inp)
with open('IntentDetection/full/lm_valid_char.npy','rb') as inp:
    valid_char = np.load(inp)

In [5]:
train_word = batchify(train_word, 25).T
train_char = batchify(train_char, 25).T
valid_word = batchify(valid_word, 21).T
valid_char = batchify(valid_char, 21).T

In [7]:
# Reset global step to zero
my_trainer.session.run(tf.assign(my_trainer.global_step, 0))

0

In [6]:
start_i = 0
lr = 5e-4
fine_tune_rate = [1/2.6, 1/3.6, 1/3.6, 1/2.6]
print(fine_tune_rate)
for _ in range(1):
    my_trainer.train_dev_loop(train_word, train_char, valid_word, valid_char, lr, start_i=start_i, folder_test='fine_tune_test', folder_train='fine_tune_train', fine_tune_rate=fine_tune_rate)
    lr = lr * 0.8
    start_i = 0

[0.3846153846153846, 0.2777777777777778, 0.2777777777777778, 0.3846153846153846]


Step 73001: progress 104/800000,forward loss 5.18940, backward loss 5.02579, ppl 165.27, bpc 07.37, time 02.76
Step 73002: progress 205/800000,forward loss 4.83607, backward loss 5.05185, ppl 140.33, bpc 07.13, time 03.59
Step 73003: progress 300/800000,forward loss 5.03948, backward loss 4.72451, ppl 131.89, bpc 07.04, time 04.33
Step 73004: progress 402/800000,forward loss 4.85868, backward loss 4.59036, ppl 112.68, bpc 06.82, time 05.12
Step 73005: progress 494/800000,forward loss 4.65357, backward loss 4.64908, ppl 104.72, bpc 06.71, time 05.95
Step 73006: progress 595/800000,forward loss 4.49658, backward loss 4.62473, ppl 95.65, bpc 06.58, time 06.54
Step 73007: progress 695/800000,forward loss 4.65610, backward loss 4.45128, ppl 94.98, bpc 06.57, time 07.28
Step 73008: progress 748/800000,forward loss 4.53977, backward loss 4.30004, ppl 83.09, bpc 06.38, time 07.79
Step 73009: progress 842/800000,forward loss 4.57180, backward loss 4.16601, ppl 78.96, bpc 06.30, time 08.46
Step 

Step 73075: progress 7223/800000,forward loss 3.57203, backward loss 3.23248, ppl 30.03, bpc 04.91, time 55.14
Step 73076: progress 7317/800000,forward loss 3.51710, backward loss 3.26257, ppl 29.66, bpc 04.89, time 55.98
Step 73077: progress 7419/800000,forward loss 3.51442, backward loss 3.12377, ppl 27.64, bpc 04.79, time 56.46
Step 73078: progress 7514/800000,forward loss 3.47183, backward loss 3.07198, ppl 26.36, bpc 04.72, time 56.92
Step 73079: progress 7613/800000,forward loss 3.60520, backward loss 3.19434, ppl 29.96, bpc 04.90, time 57.40
Step 73080: progress 7710/800000,forward loss 3.63714, backward loss 3.17352, ppl 30.12, bpc 04.91, time 58.10
Step 73081: progress 7810/800000,forward loss 3.47120, backward loss 3.08416, ppl 26.51, bpc 04.73, time 58.77
Step 73082: progress 7908/800000,forward loss 3.64459, backward loss 3.03250, ppl 28.18, bpc 04.82, time 59.25
Step 73083: progress 8009/800000,forward loss 3.55621, backward loss 2.89265, ppl 25.14, bpc 04.65, time 59.99
S

Step 73149: progress 14406/800000,forward loss 3.13381, backward loss 2.70607, ppl 18.54, bpc 04.21, time 103.32
Step 73150: progress 14502/800000,forward loss 3.14237, backward loss 2.65413, ppl 18.14, bpc 04.18, time 103.78
Step 73151: progress 14602/800000,forward loss 3.20232, backward loss 3.04262, ppl 22.70, bpc 04.50, time 104.25
Step 73152: progress 14703/800000,forward loss 3.06898, backward loss 2.79800, ppl 18.79, bpc 04.23, time 104.71
Step 73153: progress 14801/800000,forward loss 3.18156, backward loss 3.07466, ppl 22.83, bpc 04.51, time 105.22
Step 73154: progress 14914/800000,forward loss 3.24039, backward loss 2.99827, ppl 22.63, bpc 04.50, time 105.96
Step 73155: progress 15008/800000,forward loss 3.12921, backward loss 3.03752, ppl 21.83, bpc 04.45, time 106.42
Step 73156: progress 15106/800000,forward loss 3.07662, backward loss 2.93394, ppl 20.19, bpc 04.34, time 106.91
Step 73157: progress 15210/800000,forward loss 2.97409, backward loss 2.91972, ppl 19.05, bpc 04

Step 73222: progress 21296/800000,forward loss 3.06068, backward loss 2.92662, ppl 19.96, bpc 04.32, time 144.11
Step 73223: progress 21398/800000,forward loss 3.24736, backward loss 2.71581, ppl 19.72, bpc 04.30, time 144.61
Step 73224: progress 21500/800000,forward loss 3.08739, backward loss 2.87884, ppl 19.75, bpc 04.30, time 145.09
Step 73225: progress 21602/800000,forward loss 3.24649, backward loss 2.95800, ppl 22.25, bpc 04.48, time 145.60
Step 73226: progress 21652/800000,forward loss 2.98858, backward loss 3.14367, ppl 21.46, bpc 04.42, time 146.06
Step 73227: progress 21749/800000,forward loss 3.07093, backward loss 2.78225, ppl 18.66, bpc 04.22, time 146.52
Step 73228: progress 21841/800000,forward loss 3.17100, backward loss 2.75955, ppl 19.40, bpc 04.28, time 147.06
Step 73229: progress 21940/800000,forward loss 3.20863, backward loss 2.76364, ppl 19.81, bpc 04.31, time 147.81
Step 73230: progress 22047/800000,forward loss 3.33895, backward loss 2.77346, ppl 21.25, bpc 04

...

Evaluate loss 2.6259968280792236, time 633.361314535141
Evaluate loss 2.6868810653686523, time 633.6915237903595
Evaluate loss 2.6038389205932617, time 634.0310750007629
Evaluate loss 2.6386802196502686, time 634.3593280315399
Evaluate loss 2.6024656295776367, time 634.6923427581787
Evaluate loss 2.5546929836273193, time 635.0282437801361
Evaluate loss 2.64654541015625, time 635.3581187725067
Evaluate loss 2.821859121322632, time 635.6926600933075
Evaluate loss 2.6092026233673096, time 636.0237562656403
Evaluate loss 2.615420341491699, time 636.357209444046
Evaluate loss 2.6611788272857666, time 636.6913504600525
Evaluate loss 2.5583348274230957, time 637.0257794857025
Evaluate loss 2.6920056343078613, time 637.3628990650177
Evaluate loss 2.658498764038086, time 637.6954777240753
Evaluate loss 2.7430667877197266, time 638.0285105705261
Evaluate loss 2.5224061012268066, time 638.358715057373
Evaluate loss 2.5263686180114746, time 638.6926877498627
Evaluate loss 2.5928432941436768, time 

Evaluate loss 2.499819040298462, time 681.796763420105
Evaluate loss 2.348440647125244, time 682.126284122467
Evaluate loss 2.377577543258667, time 682.4517154693604
Evaluate loss 2.452148675918579, time 682.780841588974
Evaluate loss 2.543984889984131, time 683.1103632450104
Evaluate loss 2.523573398590088, time 683.4436314105988
Evaluate loss 2.5094337463378906, time 683.7757959365845
Evaluate loss 2.4781107902526855, time 684.109067440033
Evaluate loss 2.422027111053467, time 684.4374771118164
Evaluate loss 2.385645866394043, time 684.7699315547943
Evaluate loss 2.4487946033477783, time 685.1055345535278
Evaluate loss 2.495243549346924, time 685.4355046749115
Evaluate loss 2.359088182449341, time 685.7665710449219
Evaluate loss 2.4143693447113037, time 686.0995433330536
Evaluate loss 2.4174280166625977, time 686.4339747428894
Evaluate loss 2.439685344696045, time 686.7629220485687
Evaluate loss 2.3617539405822754, time 687.0935153961182
Evaluate loss 2.373802900314331, time 687.4208

Evaluate loss 2.7518506050109863, time 730.1226680278778
Evaluate loss 2.66621732711792, time 730.4559333324432
Evaluate loss 2.731321096420288, time 730.7870721817017
Evaluate loss 2.559861421585083, time 731.1265618801117
Evaluate loss 2.640178680419922, time 731.460618019104
Evaluate loss 2.605238914489746, time 731.7904298305511
Evaluate loss 2.5856268405914307, time 732.1230053901672
Evaluate loss 2.67496919631958, time 732.4533252716064
Evaluate loss 2.658921480178833, time 732.7827095985413
Evaluate loss 2.61568021774292, time 733.1122317314148
Evaluate loss 2.705820322036743, time 733.4402387142181
Evaluate loss 2.533876895904541, time 733.7690789699554
Evaluate loss 2.7587077617645264, time 734.0987541675568
Evaluate loss 2.7238240242004395, time 734.4300379753113
Evaluate loss 2.7024199962615967, time 734.7591016292572
Evaluate loss 2.57584547996521, time 735.0897109508514
Evaluate loss 2.5451910495758057, time 735.4225454330444
Evaluate loss 2.5812947750091553, time 735.7529

Evaluate loss 2.6109607219696045, time 778.6486020088196
Evaluate loss 2.6551363468170166, time 778.9931528568268
Evaluate loss 2.719953775405884, time 779.3355913162231
Evaluate loss 2.6074259281158447, time 779.6697204113007
Evaluate loss 2.643838405609131, time 779.9992480278015
Evaluate loss 2.7074930667877197, time 780.3443849086761
Evaluate loss 2.6644866466522217, time 780.6784517765045
Evaluate loss 2.6775248050689697, time 781.0110173225403
Evaluate loss 2.675278663635254, time 781.3420779705048
Evaluate loss 2.5412063598632812, time 781.6760771274567
Evaluate loss 2.633986711502075, time 782.0053153038025
Evaluate loss 2.602390766143799, time 782.3400070667267
Evaluate loss 2.664480209350586, time 782.6694238185883
Evaluate loss 2.699528455734253, time 783.0042905807495
Evaluate loss 2.5386290550231934, time 783.3398151397705
Evaluate loss 2.476621150970459, time 783.6693894863129
Evaluate loss 2.6892738342285156, time 784.0128247737885
Evaluate loss 2.5615997314453125, time 

Evaluate loss 2.4591970443725586, time 828.0392115116119
Evaluate loss 2.46545147895813, time 828.3985698223114
Evaluate loss 2.4321093559265137, time 828.7318189144135
Evaluate loss 2.4479355812072754, time 829.0664646625519
Evaluate loss 2.4914824962615967, time 829.4108345508575
Evaluate loss 2.451348066329956, time 829.7388606071472
Evaluate loss 2.607281446456909, time 830.0688030719757
Evaluate loss 2.6240880489349365, time 830.3969252109528
Evaluate loss 2.596820592880249, time 830.7472083568573
Evaluate loss 2.6636080741882324, time 831.093211889267
Evaluate loss 2.560166120529175, time 831.4387111663818
Evaluate loss 2.5153896808624268, time 831.7694704532623
Evaluate loss 2.5176970958709717, time 832.1123652458191
Evaluate loss 2.65708589553833, time 832.440999507904
Evaluate loss 2.5405728816986084, time 832.7897465229034
Evaluate loss 2.553187847137451, time 833.1477103233337
Evaluate loss 2.4921414852142334, time 833.4908993244171
Evaluate loss 2.6212875843048096, time 833